# Forcing tokens to be generated and omitted.

### GPT2

In [83]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [84]:
prompt = "Once upon a time"
bad_words = ["alcohol"]
force_words = ["fantasy", "beauty"]

In [85]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")

bad_tokens = []
for bad_word in bad_words:
    token = tokenizer.encode(bad_word, add_prefix_space=True, add_special_tokens=False)
    bad_tokens.append(token)

force_tokens = []
for force_word in force_words:
    token = tokenizer.encode(force_word, add_prefix_space=True, add_special_tokens=False)
    force_tokens.append(token)

print("bad_tokens: ", bad_tokens)
print("force_tokens: ", force_tokens)

bad_tokens:  [[5548]]
force_tokens:  [[8842], [8737]]


In [91]:
output = model.generate(
    input_ids,
    bad_words_ids=bad_tokens,
    force_words_ids=force_tokens,
    num_beams=5,
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("generated_text: ", generated_text)
print("forced all force_words_ids:", all(word in generated_text for word in force_words))
print("omitted all bad_words_ids:", all(word not in generated_text for word in bad_words))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generated_text:  Once upon a time, the beauty of this fantasy world is that it has been created by people who have never met each other. It's not like you're going to find out how they got together or what their relationship was about until later in life
forced all force_words_ids: True
omitted all bad_words_ids: True


### mT5-small-HunSum-1

In [98]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/mT5-small-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/mT5-small-HunSum-1")

In [184]:
prompt = """
A magyar szövetség vasárnapi tájékoztatása szerint a Jablonec nad Nisouban rendezett kontinensbajnokság szombati zárónapján a felnőtteknél a csoportos számban a Balázs Péter, Cséplő Kelen, Várszegi Vendel, Szász Tamás alkotta kvartett végzett bronzérmesként.

A 17 dobogós helyezéssel a magyarok másodikak lettek az éremtáblázaton

Kiss Bence csapatvezető szerint sikeres Európa-bajnokságot zártak a búvárok. Kiemelte: kiegyensúlyozott volt a válogatott, és külön öröm, hogy a felnőttek mellett a juniorok is eredményesen szerepeltek, mivel az ő eredményük is beleszámít az éremtáblázatba.

Tájékozódási búvárúszásban Monk, csillag, öt bója, szlalom, paralel és csoportos számokban rendeznek versenyeket, ezekben külön-külön bajnokokat is avatnak.
"""
bad_words = ["juniorok"]
force_words = ["bronz", "boldogak"]

In [185]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")

bad_tokens = []
for bad_word in bad_words:
    token = tokenizer.encode(bad_word, add_special_tokens=False)
    bad_tokens.append(token)

force_tokens = []
for force_word in force_words:
    token = tokenizer.encode(force_word, add_special_tokens=False)
    force_tokens.append(token)

print("bad_tokens: ", bad_tokens)
print("force_tokens: ", force_tokens)

bad_tokens:  [[43599, 925]]
force_tokens:  [[85503], [117264, 932]]


In [186]:
output = model.generate(
    input_ids,
    bad_words_ids=bad_tokens,
    force_words_ids=force_tokens,
    num_beams=2,
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    min_length=50
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("generated_text: ", generated_text)
print("forced all force_words_ids:", all(word in generated_text for word in force_words))
print("omitted all bad_words_ids:", all(word not in generated_text for word in bad_words))

generated_text:  boldogak voltak a bronzérmesek az Európa-bajnokság zárónapján - közölte Kiss Bence csapatvezetője, aki szerint kiegyensúlyozott volt az eredményük, és öröm, hogy ők is eredményesen szerepeltek.
forced all force_words_ids: True
omitted all bad_words_ids: True


### Bert2Bert-HunSum-1

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/Bert2Bert-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/Bert2Bert-HunSum-1")

Downloading:   0%|          | 0.00/413 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/266k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/952M [00:00<?, ?B/s]

In [7]:
prompt = """
A magyar szövetség vasárnapi tájékoztatása szerint a Jablonec nad Nisouban rendezett kontinensbajnokság szombati zárónapján a felnőtteknél a csoportos számban a Balázs Péter, Cséplő Kelen, Várszegi Vendel, Szász Tamás alkotta kvartett végzett bronzérmesként.

A 17 dobogós helyezéssel a magyarok másodikak lettek az éremtáblázaton

Kiss Bence csapatvezető szerint sikeres Európa-bajnokságot zártak a búvárok. Kiemelte: kiegyensúlyozott volt a válogatott, és külön öröm, hogy a felnőttek mellett a juniorok is eredményesen szerepeltek, mivel az ő eredményük is beleszámít az éremtáblázatba.

Tájékozódási búvárúszásban Monk, csillag, öt bója, szlalom, paralel és csoportos számokban rendeznek versenyeket, ezekben külön-külön bajnokokat is avatnak.
"""
bad_words = ["Gondomar"]
force_words = ["bronz", "boldogak"]

In [8]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")

bad_tokens = []
for bad_word in bad_words:
    token = tokenizer.encode(bad_word, add_special_tokens=False)
    bad_tokens.append(token)

force_tokens = []
for force_word in force_words:
    token = tokenizer.encode(force_word, add_special_tokens=False)
    force_tokens.append(token)

print("bad_tokens: ", bad_tokens)
print("force_tokens: ", force_tokens)

bad_tokens:  [[7045, 14153, 31743]]
force_tokens:  [[12820], [4800, 2032]]


In [11]:
output = model.generate(
    input_ids,
    bad_words_ids=bad_tokens,
    force_words_ids=force_tokens,
    num_beams=10,
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    min_length=50
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("generated_text: ", generated_text)
print("forced all force_words_ids:", all(word in generated_text for word in force_words))
print("omitted all bad_words_ids:", all(word not in generated_text for word in bad_words))

generated_text:  A magyar boldogak a juniorok között bronzérmet szereztek a kazanyi junior uszonyos - búvárúszó Európa - bajnokság zárónapján, miután 17 dobogós helyezéssel zárták a kontinensviadalt. A felnõtteknél pedig külön - külön bajnokot avattak.
forced all force_words_ids: True
omitted all bad_words_ids: True


## Sources to document from:
- https://huggingface.co/blog/constrained-beam-search
- https://huggingface.co/docs/transformers/v4.27.2/en/generation_strategies
- https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#generation
- https://huggingface.co/docs/transformers/v4.27.2/en/internal/generation_utils#transformers.LogitsProcessor